# 《分布式计算》2021-2022学年第2学期期末考试

提示与建议：

1. 优先完成更有把握的题目；
2. 对于简答题，回答核心要点即可，不要提供冗长的陈述；
3. 对于编程题，优先完成整体的框架，有时间富余时再进行细节上的调试；
4. 对于综合题，每一小问的题目本身可能就是对后续问题的提示；
5. 可参考课件及以往的作业，但注意与考题的差异，不要生搬硬套。

### 第1题：简答题（20分）

**(a) 结合本课程的内容，简述梯度下降法、牛顿法、L-BFGS 和 ADMM 算法各自的适用范围和优缺点。（10分）**

**(b) 在使用 Spark RDD 进行分布式计算时，是否划分的分区数越多，总的计算时间越短？有哪些可能影响计算效率的因素？（10分）**

### 第2题：编程题（30分）

In [3]:
import numpy as np
import pandas as pd
from scipy.linalg import cho_factor, cho_solve

蒙特卡洛方法的一大应用是近似计算积分。例如，如果要计算

$$
I=\int_{-\infty}^{+\infty} f(x)g(x)\mathrm{d}x,
$$

其中 $g(x)$ 是一个随机变量的密度函数，那么根据期望公式，我们有

$$
I=\mathbb{E}_{X}[f(X)],\quad X\sim g(x).
$$

根据大数定律，如果我们产生服从 $g(x)$ 分布的独立同分布随机数 $X_1,\ldots,X_n$，那么

$$
I\approx \frac{1}{n}\sum_{i=1}^n f(X_i).
$$

**(a) 根据以上信息，利用蒙特卡洛方法和 Numpy 编写程序近似计算积分**

$$I_1=\int_{-\infty}^{+\infty} \left\{[\sin(x)]^2+\cos(x)\right\}\cdot \frac{1}{\sqrt{2\pi}}e^{-x^2/2}\mathrm{d}x,$$

**生成随机数时以自己的学号作为随机数种子，取样本量 $n=10000$。（10分）**

> 提示1：标准正态分布 $N(0,1)$ 的密度函数为
>
> $$
\phi(x)=\frac{1}{\sqrt{2\pi}}e^{-\frac{x^2}{2}}.
$$

> 提示2：计算结果可以利用数值积分进行验证，例如
>
> ```py
> # 计算 h(x)=exp(-|x|) 在 (-Inf, +Inf) 上的积分
> import numpy as np
> import scipy.integrate as integrate
> 
> integrate.quad(lambda x: np.exp(-np.abs(x)), -np.inf, np.inf)
> # 结果的第一个元素是积分值，第二个是误差估计
> ```
> ```
> (2.0000000000000004, 1.1685213993527393e-10)
> ```
>
> 该程序可用来帮助验证计算结果，但无需包含在最终的回答中。

In [49]:
import math
n=10000
np.random.seed(2020110868)
dt = np.random.normal(0,1,n)

dat = sc.parallelize(dt)
dat.map(lambda x: (math.sin(x))**2+math.cos(x)).map(lambda x: (1, x)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))


(10000, 10453.899686504246)

**(b) 利用 PySpark 实现一个分布式计算 (a) 中积分的程序，原理如下：**

我们采用分布式的方法并行模拟大量的随机数。考虑将所有的随机数分成10组，其中1-5组每组生成 10000 个随机数，6-10组每组生成 5000 个点，每组独立产生随机数并计算各自的 $f(X_i)$ 求和，最后将所有10组的结果汇总并得出最终对积分的估计。为了使结果可重复，第 $i$ 组在生成随机数时使用 $i$ 作为随机数种子。PySpark 使用本地模式，开启 4 个 CPU 核心。**（10分）**

In [67]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[4]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)


import numpy as np

def MC(i):
    print(i)
    np.random.seed(i)
    if i <=5:
        dt = np.random.normal(0,1,10000)
    else:
        dt = np.random.normal(0,1,5000)
    y = np.sin(dt)**2+np.cos(dt)
    n = len(y)
    return (n,sum(y))

part=range(10)
data=sc.parallelize(part)
in_all=data.map(lambda x:MC(x)).reduce(lambda x,y:(x[0]+y[0],x[1]+y[1]))

<SparkContext master=local[8] appName=PySpark RDD>


In [68]:
in_all[1]/in_all[0]

1.0397167278478991

**(c) 利用 (b) 中的方法，近似计算积分**

$$I_2=\int_{-\infty}^{+\infty} \left\{[\sin(x)]^2+\cos(x)\right\}\cdot e^{-(x-1)^2}\mathrm{d}x.$$
**（10分）**

> 提示3：正态分布 $N(\mu,\sigma^2)$ 的密度函数为
> $$
h(x)=\frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{(x-\mu)^2}{2\sigma^2}}.
$$

In [69]:
def MC1(i):
    print(i)
    np.random.seed(i)
    if i <=5:
        dt = np.random.normal(1,math.sqrt(0.5),10000)
    else:
        dt = np.random.normal(1,math.sqrt(0.5),5000)
    y = np.sin(dt)**2+np.cos(dt)
    n = len(y)
    return (n,sum(y))

part=range(10)
data=sc.parallelize(part)
in_all=data.map(lambda x:MC1(x)).reduce(lambda x,y:(x[0]+y[0],x[1]+y[1]))
in_all[1]/in_all[0]

1.0004170710273972

### 第3题：综合题（50分）

Least Absolute Deviation (LAD) 是一种稳健回归方法，它与线性回归中的最小二乘方法（OLS）类似，但用残差的绝对值之和来替代平方和。为了与 ADMM 算法的记号匹配，我们用 $A\in\mathbb{R}^{n\times p}$ 表示自变量矩阵，$b\in\mathbb{R}^n$ 表示因变量向量，要估计的回归系数为 $x\in\mathbb{R}^p$。于是 LAD 的目标函数为

$$\Vert Ax-b\Vert_1,$$

其中 $\Vert v\Vert_1$ 表示向量 $v=(v_1,\ldots,v_n)'$ 的 $L^1$ 范数，即 $\Vert v\Vert_1=|v_1|+\cdots+|v_n|$。

LAD 可以改写为 ADMM 的形式：$f(x)=0$，$g(z)=||z||_1$，$B=-I$，$c=b$。其迭代公式为

$$
\begin{align*}
x^{k+1} & =(A'A)^{-1}A'(b+z^{k}-u^{k})\\
z^{k+1} & =S_{1/\rho}(Ax^{k+1}-b+u^{k})\\
u^{k+1} & =u^{k}+Ax^{k+1}-z^{k+1}-b,
\end{align*}
$$

其中 $S_{\kappa}(a)$ 为 soft-thresholding 运算符，定义为

$$
S_{\kappa}(a)=\begin{cases}
a-\kappa, & a>\kappa\\
0, & |a|\le\kappa\\
a+\kappa, & a<-\kappa
\end{cases},
$$

一种紧凑的表达是 $S_{\kappa}(a)=\mathrm{sign}(a)\cdot\max\{0,|a|-\kappa\}$。

相应地，原问题残差为 $r^{k+1}=Ax^{k+1}-z^{k+1}-b$，对偶问题残差为 $s^{k+1}=-\rho A'(z^{k+1}-z^{k})$。

**(a) $x$、$z$ 和 $u$ 各自的维度是多少？（3分）**

**(b) 编写函数 `update_z()` 来实现 $z$ 的更新，函数需返回更新后的 $z$ 变量。（7分）**

In [5]:
def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)
def update_z(A,b,u,rho):
    znew = soft_thresholding(A.dot(xnew) - b + u, 1.0 / rho)
    return znew

**(c) 编写函数 `update_x()` 来实现 $x$ 的更新，函数需返回更新后的 $x$ 变量。（10分）**

In [6]:
def update_x(A,b,z,u):
    xnew = np.linalg.solve(A.T.dot(A), A.T.dot(b + z - u))
    return xnew

**(d) 在 LAD 的 ADMM 算法中，$x$ 需要反复进行迭代，但注意到 $A$ 是固定的。如果假定 $A'A$ 是正定矩阵，那么应该采取怎样的方法，使得反复进行的 $x$ 更新更加高效？简述即可，无需在此处提供代码实现。（5分）**

一种更好的方法是先对  𝐴′𝐴进行 Cholesky 分解（ 𝐴′𝐴 是正定矩阵），然后再解线性方程组

**(e) 利用 ADMM 算法求解 LAD 问题**

$$\min_x\ \Vert Ax-b\Vert_1,$$

**并将其封装成一个函数：**

```python
admm_lad(A, b, rho=1.0, maxit=10000, eps=1e-3, verbose=0)
```

**其中 `A` 是 $n\times p$ 的自变量矩阵，`b` 是 $n\times 1$ 的因变量向量，`rho` 是 ADMM 算法的 $\rho$ 参数，`maxit` 是最大迭代次数，`eps` 是 ADMM 收敛的残差临界值，`verbose` 表示是否输出迭代信息，如果 $>0$，则每隔 1000 次迭代打印出当前的两类残差，如果 $\le 0$ 则不输出任何信息。**

**函数需返回两个量，第一个表示实际使用的迭代次数，第二个表示估计的回归系数。该函数可利用前面小题中已经编写的函数，也可重新编写，且应实现（d）中给出的方法。（15分）**

In [13]:
def admm_lad(A, b, rho=1.0, maxit=10000, eps=1e-3, verbose=0):
    n=A.shape[0]
    p=A.shape[1]
    x = np.zeros(p)
    z = np.zeros(n)
    u = np.zeros(n)
    c, lower = cho_factor(A.T.dot(A))
    itercount = 1
    for i in range(maxit):
        # x 更新
    #     xnew = np.linalg.solve(A.T.dot(A), A.T.dot(b + z - u))
        xnew =cho_solve((c, lower), A.T.dot(b + z - u))
        # z 更新
        Axnew = A.dot(xnew)
        znew = soft_thresholding(Axnew - b + u, 1.0 / rho)
        # u 更新
        unew = u + Axnew - znew - b
        # 计算残差大小
        resid_r_norm = np.linalg.norm(unew - u)
        resid_s_norm = rho * np.linalg.norm(A.T.dot(znew - z))
        # 更新 x、z 和 u 的取值
        x = xnew
        z = znew
        u = unew
        # 打印残差信息，判断是否收敛
        if i % 1000 == 0 and verbose >=0:
            print(f"Iteration {i}, ||r|| = {resid_r_norm:.6f}, ||s|| = {resid_s_norm:.6f}")
        if resid_r_norm <= eps and resid_s_norm <= eps:
#             break
            return i,z

利用模拟训练集数据测试上述编写的函数：

In [11]:
np.random.seed(123)
n = 1000
p = 30
Atrain = np.random.normal(size=(n, p))
x = np.random.normal(size=p)
btrain = Atrain.dot(x) + np.random.normal(size=n)
x

array([-1.05417044, -0.78301134,  1.82790084,  1.7468072 ,  1.3282585 ,
       -0.43277314, -0.6686141 , -0.47208845,  1.05554064,  0.67905585,
        0.14814832,  1.04294573,  0.28718991,  1.55577283,  0.97031604,
        0.39737593,  1.15394013, -0.00333042,  1.30948521, -0.90230241,
       -0.11259757, -2.70145552, -0.7578208 ,  0.05500775, -0.79928969,
        1.37011863, -0.42958025,  0.57753203,  0.18223881,  0.7526572 ])

In [14]:
admm_lad(Atrain, btrain, rho=1.0, maxit=10000, eps=1e-3, verbose=1)

Iteration 0, ||r|| = 22.733252, ||s|| = 35.504139
Iteration 1000, ||r|| = 0.001572, ||s|| = 0.091430
Iteration 2000, ||r|| = 0.000698, ||s|| = 0.018307
Iteration 3000, ||r|| = 0.000232, ||s|| = 0.006585
Iteration 4000, ||r|| = 0.000163, ||s|| = 0.003721
Iteration 5000, ||r|| = 0.000094, ||s|| = 0.004071


(5448,
 array([-3.30954260e-01, -1.35236637e+00,  2.08158513e+00, -0.00000000e+00,
        -1.83756877e-01,  1.99796012e+00, -7.22794153e-01,  1.06680710e+00,
        -9.11602263e-01, -2.21894577e+00,  1.56448132e+00,  0.00000000e+00,
         1.34099834e-01,  3.12968133e-02, -3.64175452e-01, -3.54873873e-02,
         3.56243831e-02,  1.91609722e+00, -2.94350923e-01, -1.13897819e+00,
         2.37450751e-01, -6.16053926e-01,  1.18789167e+00, -1.24677130e+00,
        -2.24693817e+00, -1.19588424e+00,  1.26084818e-01, -5.04550675e-01,
        -1.37088790e+00, -1.26102852e+00,  1.14127458e-01, -2.31540490e+00,
         1.28355273e+00,  6.20475951e-01,  1.27852455e+00, -7.28239476e-01,
         3.08316154e-01,  8.28830665e-02,  1.46298373e+00, -1.00019037e+00,
         1.17248113e+00, -1.58858362e+00,  3.61652605e-01,  1.24492052e+00,
        -7.32213530e-01, -3.90377552e-01, -0.00000000e+00, -1.50870761e+00,
         1.53308880e-01, -1.01028681e+00, -1.29077504e-01, -2.02226439e-01,
     

In [15]:
admm_lad(Atrain, btrain, rho=0.1, maxit=10000, eps=1e-3, verbose=0)

Iteration 0, ||r|| = 31.253316, ||s|| = 0.000000
Iteration 1000, ||r|| = 0.011995, ||s|| = 0.031087
Iteration 2000, ||r|| = 0.003671, ||s|| = 0.021444
Iteration 3000, ||r|| = 0.002559, ||s|| = 0.009755
Iteration 4000, ||r|| = 0.000803, ||s|| = 0.008039
Iteration 5000, ||r|| = 0.001406, ||s|| = 0.004060
Iteration 6000, ||r|| = 0.001255, ||s|| = 0.001987
Iteration 7000, ||r|| = 0.000979, ||s|| = 0.001314


(7125,
 array([-3.31298035e-01, -1.35359531e+00,  2.08170944e+00, -0.00000000e+00,
        -1.84320949e-01,  1.99788183e+00, -7.22134688e-01,  1.06576668e+00,
        -9.12282753e-01, -2.21942659e+00,  1.56507949e+00,  0.00000000e+00,
         1.34319569e-01,  3.07719026e-02, -3.63611026e-01, -3.47538086e-02,
         3.71840824e-02,  1.91507033e+00, -2.93112580e-01, -1.13901019e+00,
         2.37327135e-01, -6.14393270e-01,  1.18706971e+00, -1.24666543e+00,
        -2.24862890e+00, -1.19543812e+00,  1.26938698e-01, -5.04561562e-01,
        -1.37048575e+00, -1.26151598e+00,  1.14659920e-01, -2.31598582e+00,
         1.28538601e+00,  6.20205284e-01,  1.27908966e+00, -7.26659750e-01,
         3.07314112e-01,  8.16083641e-02,  1.46216311e+00, -9.99574760e-01,
         1.17243175e+00, -1.58828651e+00,  3.61280198e-01,  1.24366296e+00,
        -7.31922497e-01, -3.90122469e-01, -0.00000000e+00, -1.50907768e+00,
         1.53282542e-01, -1.01160460e+00, -1.28083028e-01, -2.02403913e-01,
     

**(f) 利用 PySpark 来并行地考察 $\rho$ 参数对算法收敛速度的影响。取 $\rho=0.1,0.2,\ldots,1.0$，找出使得迭代次数最少的 $\rho$。（10分）**

In [17]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[8]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext

In [28]:
params=[]
for i in range(1,11):
    params.append(i/10)
params_data=sc.parallelize(params)

In [29]:
def answer(x):
    niter=admm_lad(Atrain, btrain, rho=x, maxit=10000, eps=1e-3, verbose=0)[0]
#     mse=sum((Xtest.dot(yhat)-ytest)**2)/ntest
#     return f'niter={niter}'
    return niter

In [31]:
s = params_data.map(lambda x:answer(x)).collect()

In [35]:
params[np.argmin(s)]

0.4